# **1. Filtrando Ações e df ex_R_acoes**

In [1]:
# === Carregar e filtrar ações com log-returns mensais completos (sem NA) ===
from pathlib import Path
import pandas as pd

# Caminho do arquivo
xlsx_path = Path("Base_Final_DQAI2025_v3.xlsx")

# 1) Ler a sheet "Acoes Log Ret M"
#    - Converte a coluna "Data" para datetime (formato dd/mm/yyyy hh:mm:ss)
#    - Mantém a ordem mensal EOM
df_raw = pd.read_excel(
    xlsx_path,
    sheet_name="Acoes Log Ret M"
)

# Garantir que a coluna 'Data' exista
assert "Data" in df_raw.columns, "A sheet 'Acoes Log Ret M' deve conter a coluna 'Data'."

# 2) Tratar a coluna de datas
df_raw["Data"] = pd.to_datetime(df_raw["Data"], dayfirst=True, errors="coerce")
# Remover linhas com Data inválida (se houver)
df_raw = df_raw.dropna(subset=["Data"]).sort_values("Data").reset_index(drop=True)

# 3) Separar colunas
date_col = ["Data"]
action_cols = [c for c in df_raw.columns if c != "Data"]

# (Opcional, robustez) Assegurar que as colunas de ações sejam numéricas
# Caso haja strings acidentais, isso converte para numérico e introduz NaN onde não for número
df_actions = df_raw[action_cols].apply(pd.to_numeric, errors="coerce")

# 4) Remover colunas com qualquer NA ao longo da amostra
mask_full = ~df_actions.isna().any(axis=0)
full_cols = [col for col in df_actions.columns if mask_full[col]]

# 5) Montar o DataFrame final
df_clean = pd.concat([df_raw[date_col], df_actions[full_cols]], axis=1)

# 6) Quantidade de ações remanescentes
n_actions = len(full_cols)

# 7) Prints de checagem
print(f"Período: {df_clean['Data'].min().date()} → {df_clean['Data'].max().date()} | n_meses = {df_clean.shape[0]}")
print(f"Ações com série completa (sem NA) em toda a amostra: {n_actions}")
if n_actions > 0:
    preview = ", ".join(full_cols[:10])
    extra = "" if n_actions <= 10 else f" ... (+{n_actions-10} mais)"
    print(f"Exemplos de tickers: {preview}{extra}")
else:
    print("Nenhuma ação possui série completa. Verifique a consistência da base ou relaxe o critério.")

Período: 2013-12-31 → 2024-12-31 | n_meses = 133
Ações com série completa (sem NA) em toda a amostra: 114
Exemplos de tickers: ABCB4, ABEV3, AGRO3, ALPA4, ALUP11, AMER3, ANIM3, AZZA3, B3SA3, BBAS3 ... (+104 mais)


In [2]:
import pandas as pd
from pathlib import Path

# --- Pré-requisito: df_clean já existe em memória ---
# df_clean: Data + 114 ações sem NA (criado no passo anterior)

xlsx_path = Path("Base_Final_DQAI2025_v3.xlsx")

# 1) Ler a sheet de CDI (log return mensal)
df_cdi = pd.read_excel(xlsx_path, sheet_name="CDI Log Ret M")

# 2) Tratar datas e identificar a coluna do CDI (primeira não-'Data')
assert "Data" in df_cdi.columns, "A sheet 'CDI Log Ret M' deve conter a coluna 'Data'."
df_cdi["Data"] = pd.to_datetime(df_cdi["Data"], dayfirst=True, errors="coerce")
df_cdi = df_cdi.dropna(subset=["Data"]).sort_values("Data").reset_index(drop=True)

cdi_cols = [c for c in df_cdi.columns if c != "Data"]
assert len(cdi_cols) >= 1, "A sheet 'CDI Log Ret M' precisa ter ao menos 1 coluna de CDI além de 'Data'."
cdi_col = cdi_cols[0]  # usa a primeira coluna de CDI disponível

# Garantir numérico
df_cdi[cdi_col] = pd.to_numeric(df_cdi[cdi_col], errors="coerce")

# 3) Alinhar datas com df_clean (inner join)
df_merged = df_clean.merge(df_cdi[["Data", cdi_col]], on="Data", how="inner")

# 4) Calcular excesso de retorno log para cada ação
action_cols = [c for c in df_clean.columns if c != "Data"]
cdi_series = df_merged[cdi_col]

excess_values = df_merged[action_cols].sub(cdi_series, axis=0)

# 5) Montar o DataFrame final: Data + 114 colunas de excessos
ex_R_acoes = pd.concat([df_merged[["Data"]], excess_values], axis=1)

# 6) Checagens
print(f"ex_R_acoes: {ex_R_acoes.shape[0]} meses × {ex_R_acoes.shape[1]-1} ações (excessos)")
print(f"Período alinhado: {ex_R_acoes['Data'].min().date()} → {ex_R_acoes['Data'].max().date()}")
print(f"Coluna de CDI utilizada: {cdi_col}")

# (Opcional) Salvar em disco
ex_R_acoes.to_parquet("ex_R_acoes.parquet", index=False)
ex_R_acoes.to_csv("ex_R_acoes.csv", index=False)

ex_R_acoes: 133 meses × 114 ações (excessos)
Período alinhado: 2013-12-31 → 2024-12-31
Coluna de CDI utilizada: CDI_LogRet_M


# **2. Random Fourier Features (RFFs)**

In [81]:
import pandas as pd
from pathlib import Path

# Caminho do arquivo
xlsx_path = Path("Base_Final_DQAI2025_v3.xlsx")

# 1) Ler a sheet de fatores mensais (Bryan Kelly Brasil)
#    OBS: o nome da aba é "Fatores BK M" (ajuste se estiver diferente no seu arquivo)
df_fatores = pd.read_excel(
    xlsx_path,
    sheet_name="Fatores BK M"
)

# 2) Tratar coluna de datas
assert "Data" in df_fatores.columns, "A sheet 'Fatoes BK M' deve conter a coluna 'Data'."

df_fatores["Data"] = pd.to_datetime(df_fatores["Data"], dayfirst=True, errors="coerce")
df_fatores = (
    df_fatores
    .dropna(subset=["Data"])
    .sort_values("Data")
    .reset_index(drop=True)
)

# 3) Separar colunas de fatores (148 colunas além de 'Data')
date_col = "Data"
factor_cols = [c for c in df_fatores.columns if c != date_col]

# Garantir que todos os fatores estão em formato numérico
df_X = df_fatores[factor_cols].apply(pd.to_numeric, errors="coerce")

# 4) Calcular desvio padrão expansivo com mínimo de 24 meses
#    expanding().std() usa ddof=1 (desvio padrão amostral), o que é ok para escala
rolling_std = df_X.expanding(min_periods=24).std()

# 5) Padronizar: fator / desvio padrão expansivo
df_X_std = df_X / rolling_std

# 6) Montar o DataFrame final G_t:
#    - Mantemos a coluna 'Data'
#    - Removemos as linhas iniciais onde ainda não havia 24 meses (std = NaN)
df_Gt = pd.concat([df_fatores[[date_col]], df_X_std], axis=1)

# Eliminar linhas com qualquer NaN nos fatores (período antes dos 24 meses)
mask_valid = ~df_Gt[factor_cols].isna().any(axis=1)
df_Gt = df_Gt.loc[mask_valid].reset_index(drop=True)

# 7) Checagens rápidas
n_meses_original = df_fatores.shape[0]
n_meses_Gt = df_Gt.shape[0]

print(f"Meses na base original de fatores: {n_meses_original}")
print(f"Meses após padronização expansiva (mínimo 24 meses): {n_meses_Gt}")
print(f"Período em G_t: {df_Gt['Data'].min().date()} → {df_Gt['Data'].max().date()}")
print(f"Número de fatores em G_t: {len(factor_cols)}")

# (Opcional) salvar em disco para reutilizar depois
df_Gt.to_parquet("Gt_fatoresBK_expanding24.parquet", index=False)
df_Gt.to_csv("Gt_fatoresBK_expanding24.csv", index=False)

Meses na base original de fatores: 133
Meses após padronização expansiva (mínimo 24 meses): 110
Período em G_t: 2015-11-30 → 2024-12-31
Número de fatores em G_t: 148


In [157]:
import numpy as np
import pandas as pd
from pathlib import Path

# =========================================================
# 1) Carregar G_t padronizado (se df_Gt já estiver em memória, pode pular o read)
# =========================================================
# Se você já tem df_Gt carregado, comente as duas linhas abaixo
gt_path = Path("Gt_fatoresBK_expanding24.parquet")  # ajuste se usou outro nome
df_Gt = pd.read_parquet(gt_path)

# Checar estrutura
assert "Data" in df_Gt.columns, "df_Gt deve conter a coluna 'Data'."
factor_cols = [c for c in df_Gt.columns if c != "Data"]
print(f"Nº de meses em G_t: {df_Gt.shape[0]}, Nº de fatores: {len(factor_cols)}")

# =========================================================
# 2) Preparar matriz de fatores G (T x d)
# =========================================================
G = df_Gt[factor_cols].to_numpy(dtype=float)   # shape (T, 148)
T, d = G.shape
print(f"Shape de G: {G.shape} (T x d)")

# =========================================================
# 3) Gerar Random Fourier Features
#    - P = 12.000 (6.000 senos + 6.000 cossenos)
#    - γ = 2
#    - seed fixa = 123
# =========================================================
P = 12000
half_P = P // 2
gamma = 1.0

rng = np.random.default_rng(123)

# Ω ~ N(0, I_d), com shape (d, 6000)
Omega = rng.normal(loc=0.0, scale=1.0, size=(d, half_P))

# Projeções lineares: A = G @ Ω  → shape (T, 6000)
A = G @ Omega

# RFF:
# Primeiro 6000 colunas: sin(γ * A)
# Próximas 6000:        cos(γ * A)
S_sin = np.sin(gamma * A)         # (T, 6000)
S_cos = np.cos(gamma * A)         # (T, 6000)

# Concatenar para formar 12.000 features
S_full = np.concatenate([S_sin, S_cos], axis=1)   # shape (T, 12000)

print(f"Shape de S_full (RFF): {S_full.shape} (T x P)")

# =========================================================
# 4) Montar df_St: Data + 12.000 RFF
# =========================================================
rff_cols = [f"RFF_{k+1}" for k in range(P)]  # RFF_1, ..., RFF_12000

df_St = pd.DataFrame(S_full, columns=rff_cols)
df_St.insert(0, "Data", df_Gt["Data"].values)

# Checagens
print(df_St.head())
print(f"df_St: {df_St.shape[0]} meses × {df_St.shape[1]-1} RFFs")

# (Opcional) salvar para reutilizar depois
df_St.to_parquet("St_RFF_12000_g2_seed123.parquet", index=False)
df_St.to_csv("St_RFF_12000_g2_seed123.csv", index=False)

Nº de meses em G_t: 110, Nº de fatores: 148
Shape de G: (110, 148) (T x d)
Shape de S_full (RFF): (110, 12000) (T x P)
        Data     RFF_1     RFF_2     RFF_3     RFF_4     RFF_5     RFF_6  \
0 2015-11-30  0.864096  0.674262 -0.686221  0.950084 -0.998861 -0.372674   
1 2015-12-31 -0.805667  0.500175 -0.645283 -0.518692 -0.945397 -0.984444   
2 2016-01-31  0.442689  0.813684  0.211457 -0.909946  0.361474  0.996318   
3 2016-02-29 -0.652902 -0.089506 -0.109048  0.832404  0.504901  0.098547   
4 2016-03-31 -0.378944  0.857365 -0.244482  0.262824  0.954738 -0.696470   

      RFF_7     RFF_8     RFF_9  ...  RFF_11991  RFF_11992  RFF_11993  \
0  0.974051 -0.193734 -0.192409  ...  -0.798717  -0.947031   0.945496   
1 -0.377093 -0.490091 -0.989208  ...  -0.822754  -0.279857  -0.334332   
2  0.829912 -0.900614 -0.997620  ...  -0.538275  -0.926270   0.999826   
3 -0.806354 -0.768842 -0.577185  ...   0.970832  -0.461626  -0.736420   
4  0.981423 -0.971630 -0.142928  ...   0.111635  -0.629271 

In [105]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path

# ================================
# 1) Carregar df_St (ajuste o nome se precisar)
# ================================
base_dir = Path(".")  # diretório atual do seu projeto

# Ajuste aqui se você usou outro nome
parquet_path = base_dir / "St_RFF_12000_g2_seed123.parquet"
csv_path = base_dir / "St_RFF_12000_g2_seed123.csv"

if parquet_path.exists():
    df_St = pd.read_parquet(parquet_path)
elif csv_path.exists():
    df_St = pd.read_csv(csv_path, parse_dates=["Data"])
else:
    raise FileNotFoundError("Nem parquet nem csv de df_St foram encontrados no diretório atual.")

print(f"df_St carregado: {df_St.shape[0]} meses × {df_St.shape[1]-1} RFFs")

# ================================
# 2) Selecionar 100 primeiras RFFs
# ================================
rff_cols = [c for c in df_St.columns if c.startswith("RFF_")]
rff_cols_100 = rff_cols[:100]

df_subset = df_St[rff_cols_100]

# ================================
# 3) Matriz de correlação (|corr|)
# ================================
corr_matrix = df_subset.corr().abs()

# ================================
# 4) Colormap roxo: #F0DFFF → #220042
# ================================
colors = ["#B98DE2", "#220042"]
purple_cmap = LinearSegmentedColormap.from_list("custom_purple", colors)

# ================================
# 5) Plot do heatmap
# ================================
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(corr_matrix, cmap=purple_cmap, vmin=0, vmax=1)

num_features = len(rff_cols_100)
tick_positions = np.arange(0, num_features, 10)
tick_labels = [rff_cols_100[i] for i in tick_positions]

ax.set_xticks(tick_positions)
ax.set_yticks(tick_positions)
ax.set_xticklabels(tick_labels, rotation=90)
ax.set_yticklabels(tick_labels)

ax.set_title("Heatmap de Correlação |RFF| (100 primeiras features)")
cbar = fig.colorbar(im, ax=ax)
cbar.set_label("|Correlação|")

plt.tight_layout()

# ================================
# 6) Criar pasta e salvar imagem
# ================================
output_dir = base_dir / "Gráficos de Preparação"
os.makedirs(output_dir, exist_ok=True)

output_path = output_dir / "heatmapRFF.png"
fig.savefig(output_path, dpi=300, bbox_inches="tight")
plt.close(fig)

print(f"Heatmap salvo em: {output_path}")

df_St carregado: 110 meses × 6000 RFFs
Heatmap salvo em: Gráficos de Preparação\heatmapRFF.png


# **3. Excesso de Retornos $t + 1$**

In [158]:
import pandas as pd
from pathlib import Path

# ---------------------------------------------
# 1) Carregar df_St (apenas para pegar as datas)
# ---------------------------------------------
st_path_parquet = Path("St_RFF_12000_g2_seed123.parquet")
st_path_csv     = Path("St_RFF_12000_g2_seed123.csv")

if st_path_parquet.exists():
    df_St = pd.read_parquet(st_path_parquet)
elif st_path_csv.exists():
    df_St = pd.read_csv(st_path_csv, parse_dates=["Data"])
else:
    raise FileNotFoundError("Não encontrei df_St (parquet/csv) no diretório.")

# Conferência rápida
assert "Data" in df_St.columns, "df_St precisa ter coluna 'Data'."

# ---------------------------------------------
# 2) Carregar excessos de retorno em t (ex_R_acoes)
#    (ou use diretamente o df que você já criou em memória)
# ---------------------------------------------
ex_path_parquet = Path("ex_R_acoes.parquet")
ex_path_csv     = Path("ex_R_acoes.csv")

if ex_path_parquet.exists():
    ex_R_acoes = pd.read_parquet(ex_path_parquet)
elif ex_path_csv.exists():
    ex_R_acoes = pd.read_csv(ex_path_csv, parse_dates=["Data"])
else:
    raise FileNotFoundError("Não encontrei ex_R_acoes (parquet/csv) no diretório.")

assert "Data" in ex_R_acoes.columns, "ex_R_acoes precisa ter coluna 'Data'."

# Ordenar por segurança
ex_R_acoes = ex_R_acoes.sort_values("Data").reset_index(drop=True)

# ---------------------------------------------
# 3) Shift de +1 mês para construir alvo t+1
#    (trazer ex_R_acoes(t+1) para a linha de t)
# ---------------------------------------------
# Colunas das ações (todas menos 'Data')
action_cols = [c for c in ex_R_acoes.columns if c != "Data"]

# Copiamos para não mexer no original
ex_R_acoes_tmais1 = ex_R_acoes.copy()

# Shift de -1 nas colunas das ações:
# linha t passa a conter o retorno de t+1
ex_R_acoes_tmais1[action_cols] = ex_R_acoes_tmais1[action_cols].shift(-1)

# ---------------------------------------------
# 4) Alinhar com as datas de df_St
#    (Data igual à de df_St, valores = excessos de t+1)
# ---------------------------------------------
df_datas_St = df_St[["Data"]].copy()

# Merge por Data; left para manter exatamente as datas de df_St
ex_R_acoes_tmais1_treino = df_datas_St.merge(
    ex_R_acoes_tmais1,
    on="Data",
    how="left"
)

# Agora ex_R_acoes_tmais1_treino tem:
# - Data (mesmas 110 datas de df_St)
# - 114 colunas com ex_R de t+1 (última linha deve ficar NaN)

# ---------------------------------------------
# 5) Checagens
# ---------------------------------------------
print(ex_R_acoes_tmais1_treino.head())
print(ex_R_acoes_tmais1_treino.tail())
print(
    f"Shape ex_R_acoes_tmais1_treino: "
    f"{ex_R_acoes_tmais1_treino.shape[0]} meses × "
    f"{ex_R_acoes_tmais1_treino.shape[1]-1} ações"
)

na_ultima_linha = ex_R_acoes_tmais1_treino.iloc[-1][action_cols].isna().all()
print(f"Última linha com NaN em todas as ações (esperado): {na_ultima_linha}")

# (Opcional) Salvar em disco
ex_R_acoes_tmais1_treino.to_parquet("ex_R_acoes_tmais1_treino.parquet", index=False)
ex_R_acoes_tmais1_treino.to_csv("ex_R_acoes_tmais1_treino.csv", index=False)

        Data     ABCB4     ABEV3     AGRO3     ALPA4    ALUP11     AMER3  \
0 2015-11-30 -0.058535 -0.046484  0.033895 -0.066233 -0.086251  0.019800   
1 2015-12-31 -0.028165  0.033885 -0.159821  0.013102 -0.070833 -0.149009   
2 2016-01-31  0.079115 -0.059187  0.150183  0.188459  0.008443 -0.148032   
3 2016-02-29  0.323309  0.053213  0.001825 -0.142786  0.151971  0.202036   
4 2016-03-31 -0.047761  0.019348  0.049614  0.166441 -0.004281 -0.053353   

      ANIM3     AZZA3     B3SA3  ...     TUPY3     UCAS3     UGPA3     UNIP6  \
0  0.083402 -0.049266 -0.033412  ... -0.078255  0.099524 -0.048569  0.103354   
1 -0.334579 -0.110577 -0.070086  ... -0.062069 -0.039995 -0.028353 -0.055366   
2  0.157551 -0.074644  0.114510  ... -0.008277  0.036827  0.059749  0.150378   
3 -0.182065  0.259298  0.268802  ... -0.086815  0.170783  0.093201  0.218985   
4  0.060466  0.027330  0.100189  ... -0.200110  0.040566  0.026502  0.029548   

      USIM3     USIM5     VALE3     VLID3     WEGE3     YDUQ3 

# **4. Mapeando as Janelas de Treino de 12M (Evitar Contaminação)**

In [159]:
import pandas as pd
from pathlib import Path

# ================================
# 1) Carregar df_St (features RFF)
# ================================
st_path_parquet = Path("St_RFF_12000_g2_seed123.parquet")
st_path_csv     = Path("St_RFF_12000_g2_seed123.csv")

if st_path_parquet.exists():
    df_St = pd.read_parquet(st_path_parquet)
elif st_path_csv.exists():
    df_St = pd.read_csv(st_path_csv, parse_dates=["Data"])
else:
    raise FileNotFoundError("Não encontrei df_St (parquet/csv) no diretório.")

assert "Data" in df_St.columns, "df_St precisa ter coluna 'Data'."
df_St = df_St.sort_values("Data").reset_index(drop=True)

# ================================
# 2) Carregar ex_R_acoes_tmais1_treino (targets t+1)
# ================================
ex_t1_path_parquet = Path("ex_R_acoes_tmais1_treino.parquet")
ex_t1_path_csv     = Path("ex_R_acoes_tmais1_treino.csv")

if ex_t1_path_parquet.exists():
    ex_R_acoes_tmais1_treino = pd.read_parquet(ex_t1_path_parquet)
elif ex_t1_path_csv.exists():
    ex_R_acoes_tmais1_treino = pd.read_csv(ex_t1_path_csv, parse_dates=["Data"])
else:
    raise FileNotFoundError("Não encontrei ex_R_acoes_tmais1_treino (parquet/csv) no diretório.")

assert "Data" in ex_R_acoes_tmais1_treino.columns, "ex_R_acoes_tmais1_treino precisa ter coluna 'Data'."
ex_R_acoes_tmais1_treino = ex_R_acoes_tmais1_treino.sort_values("Data").reset_index(drop=True)

# ================================
# 3) Garantir alinhamento de datas
# ================================
# Aqui assumimos que df_St e ex_R_acoes_tmais1_treino têm o mesmo calendário
if not df_St["Data"].equals(ex_R_acoes_tmais1_treino["Data"]):
    raise ValueError("As datas de df_St e ex_R_acoes_tmais1_treino não estão perfeitamente alinhadas.")

datas = df_St["Data"].copy()
n_rows = len(datas)

# Colunas de ações (targets)
action_cols = [c for c in ex_R_acoes_tmais1_treino.columns if c != "Data"]

# ================================
# 4) Construir o "mapa de janelas"
# ================================
janela_treino = 12

rows = []
for idx_pred in range(n_rows):
    # Precisamos de 12 meses ANTES de idx_pred: [idx_pred-12, ..., idx_pred-1]
    idx_train_start = idx_pred - janela_treino
    idx_train_end   = idx_pred - 1

    # Se não tem 12 meses antes, pula (ainda estamos no burn-in)
    if idx_train_start < 0:
        continue

    # Se os targets nessa linha são todos NaN, não nos interessa como previsão
    row_targets = ex_R_acoes_tmais1_treino.loc[idx_pred, action_cols]
    if row_targets.isna().all():
        continue

    rows.append({
        "idx_pred": idx_pred,
        "Data_pred": datas.iloc[idx_pred],
        "idx_train_start": idx_train_start,
        "idx_train_end": idx_train_end
    })

window_map = pd.DataFrame(rows)

# ================================
# 5) Checagens
# ================================
print(window_map.head())
print(window_map.tail())
print(f"Número de janelas de treino (previsões OOS): {window_map.shape[0]}")

# Verificar primeira janela
first = window_map.iloc[0]
print("\nPrimeira janela:")
print(f"  Data_pred          = {first['Data_pred'].date()}")
print(f"  idx_pred           = {first['idx_pred']}")
print(f"  idx_train_start    = {first['idx_train_start']}")
print(f"  idx_train_end      = {first['idx_train_end']}")
print(f"  Data treino início = {datas.iloc[first['idx_train_start']].date()}")
print(f"  Data treino fim    = {datas.iloc[first['idx_train_end']].date()}")

# (Opcional) salvar o mapa de janelas
window_map.to_parquet("window_map_12m.parquet", index=False)
window_map.to_csv("window_map_12m.csv", index=False)

   idx_pred  Data_pred  idx_train_start  idx_train_end
0        12 2016-11-30                0             11
1        13 2016-12-31                1             12
2        14 2017-01-31                2             13
3        15 2017-02-28                3             14
4        16 2017-03-31                4             15
    idx_pred  Data_pred  idx_train_start  idx_train_end
92       104 2024-07-31               92            103
93       105 2024-08-31               93            104
94       106 2024-09-30               94            105
95       107 2024-10-31               95            106
96       108 2024-11-30               96            107
Número de janelas de treino (previsões OOS): 97

Primeira janela:
  Data_pred          = 2016-11-30
  idx_pred           = 12
  idx_train_start    = 0
  idx_train_end      = 11
  Data treino início = 2015-11-30
  Data treino fim    = 2016-10-31


In [160]:
import numpy as np
import pandas as pd
from pathlib import Path


# ============================================================
# 1) Carregar bases principais: df_St, ex_R_acoes_tmais1_treino, window_map
# ============================================================

# Ajuste os nomes se você salvou com outros
st_path_parquet   = Path("St_RFF_12000_g2_seed123.parquet")
st_path_csv       = Path("St_RFF_12000_g2_seed123.csv")

ex_t1_path_parquet = Path("ex_R_acoes_tmais1_treino.parquet")
ex_t1_path_csv     = Path("ex_R_acoes_tmais1_treino.csv")

window_map_parquet = Path("window_map_12m.parquet")
window_map_csv     = Path("window_map_12m.csv")

# --- df_St: RFF brutas ---
if st_path_parquet.exists():
    df_St = pd.read_parquet(st_path_parquet)
elif st_path_csv.exists():
    df_St = pd.read_csv(st_path_csv, parse_dates=["Data"])
else:
    raise FileNotFoundError("Não encontrei df_St (parquet/csv). Verifique o caminho/nome do arquivo.")

assert "Data" in df_St.columns, "df_St precisa ter coluna 'Data'."
df_St = df_St.sort_values("Data").reset_index(drop=True)

# --- ex_R_acoes_tmais1_treino: excessos t+1 ---
if ex_t1_path_parquet.exists():
    ex_R_acoes_tmais1_treino = pd.read_parquet(ex_t1_path_parquet)
elif ex_t1_path_csv.exists():
    ex_R_acoes_tmais1_treino = pd.read_csv(ex_t1_path_csv, parse_dates=["Data"])
else:
    raise FileNotFoundError("Não encontrei ex_R_acoes_tmais1_treino (parquet/csv).")

assert "Data" in ex_R_acoes_tmais1_treino.columns, "ex_R_acoes_tmais1_treino precisa ter coluna 'Data'."
ex_R_acoes_tmais1_treino = ex_R_acoes_tmais1_treino.sort_values("Data").reset_index(drop=True)

# --- window_map: índices de treino e previsão ---
if window_map_parquet.exists():
    window_map = pd.read_parquet(window_map_parquet)
elif window_map_csv.exists():
    window_map = pd.read_csv(window_map_csv, parse_dates=["Data_pred"])
else:
    raise FileNotFoundError("Não encontrei window_map_12m (parquet/csv).")

# ============================================================
# 2) Checagens básicas e definição de colunas
# ============================================================

# Verificar alinhamento total de datas entre df_St e ex_R_acoes_tmais1_treino
if not df_St["Data"].equals(ex_R_acoes_tmais1_treino["Data"]):
    raise ValueError("As datas de df_St e ex_R_acoes_tmais1_treino não estão alinhadas 1:1.")

datas = df_St["Data"].copy()
n_rows = len(datas)

# Colunas de RFF (features) e de ações (targets)
rff_cols = [c for c in df_St.columns if c.startswith("RFF_")]
action_cols = [c for c in ex_R_acoes_tmais1_treino.columns if c != "Data"]

print(f"df_St: {df_St.shape[0]} meses × {len(rff_cols)} RFFs")
print(f"ex_R_acoes_tmais1_treino: {ex_R_acoes_tmais1_treino.shape[0]} meses × {len(action_cols)} ações")
print(f"window_map com {window_map.shape[0]} janelas de treino (previsões OOS)")


# ============================================================
# 3) Função para obter os dados de UMA janela de treino
#    Ainda sem Ridge: apenas separa e padroniza X (RFF) por janela
# ============================================================

def prepare_window_data(
    df_St: pd.DataFrame,
    ex_R_t1: pd.DataFrame,
    window_map: pd.DataFrame,
    window_idx: int,
    rff_cols: list,
    action_cols: list,
    std_epsilon: float = 1e-8
):
    """
    Prepara os dados de UMA janela de treino (k-ésima linha de window_map).

    Retorna um dicionário com:
      - 'Data_pred'   : data da previsão (t)
      - 'idx_pred'    : índice da linha t
      - 'train_slice' : (idx_train_start, idx_train_end)

      - 'X_train'     : RFF padronizadas na janela de treino (T_window × P)
      - 'X_pred'      : RFF padronizadas no ponto de previsão (1 × P)

      - 'Y_train'     : excessos t+1 na janela de treino (T_window × N_ativos)
      - 'y_real'      : excesso t+1 realizado no mês previsto (1 × N_ativos)

      - 'rff_cols'    : lista de colunas usadas em X
      - 'action_cols' : lista de colunas de ações (ordem de Y)
    """
    row = window_map.iloc[window_idx]
    idx_pred = int(row["idx_pred"])
    idx_train_start = int(row["idx_train_start"])
    idx_train_end = int(row["idx_train_end"])
    data_pred = row["Data_pred"]

    # ---------- Features (RFF) bruto ----------
    # Usar .iloc porque idx_* são índices posicionais (0..n-1)
    S_window_raw = df_St.loc[idx_train_start:idx_train_end, rff_cols].to_numpy(dtype=float)
    S_pred_raw = df_St.loc[[idx_pred], rff_cols].to_numpy(dtype=float)  # matriz 1 × P

    # ---------- Targets (excessos t+1) ----------
    Y_window = ex_R_t1.loc[idx_train_start:idx_train_end, action_cols].to_numpy(dtype=float)
    y_real = ex_R_t1.loc[[idx_pred], action_cols].to_numpy(dtype=float)  # 1 × N_ativos

    # ---------- Padronização das RFF na janela ----------
    # média e desvio da janela de treino (eixo 0 = colunas)
    mean = S_window_raw.mean(axis=0)
    std = S_window_raw.std(axis=0, ddof=0)  # ddof=0 (pop), ddof=1 também seria aceitável

    # tratar std = 0 (feature constante na janela)
    std_safe = np.where(std < std_epsilon, 1.0, std)

    # padronizar
    X_train = (S_window_raw - mean) / std_safe
    X_pred = (S_pred_raw - mean) / std_safe

    # Checagens rápidas (opcional, pode comentar depois)
    if np.isnan(X_train).any() or np.isinf(X_train).any():
        raise ValueError(f"X_train contém NaN/inf na janela {window_idx}. Verificar.")
    if np.isnan(X_pred).any() or np.isinf(X_pred).any():
        raise ValueError(f"X_pred contém NaN/inf na janela {window_idx}. Verificar.")

    return {
        "Data_pred": data_pred,
        "idx_pred": idx_pred,
        "train_slice": (idx_train_start, idx_train_end),

        "X_train": X_train,          # (T_window × P)
        "X_pred": X_pred,            # (1 × P)
        "Y_train": Y_window,         # (T_window × N_ativos)
        "y_real": y_real,            # (1 × N_ativos)

        "rff_cols": rff_cols,
        "action_cols": action_cols,
    }


# ============================================================
# 4) (Opcional) Exemplo de uso para verificar se tudo está OK
#    Ainda sem Ridge: só inspecionando shapes e estatísticas básicas
# ============================================================

# Pega a primeira janela como exemplo
example_idx = 0
example = prepare_window_data(
    df_St=df_St,
    ex_R_t1=ex_R_acoes_tmais1_treino,
    window_map=window_map,
    window_idx=example_idx,
    rff_cols=rff_cols,
    action_cols=action_cols
)

print("\n=== Exemplo da primeira janela ===")
print("Data_pred:", example["Data_pred"])
print("idx_pred:", example["idx_pred"])
print("train_slice (start, end):", example["train_slice"])
print("X_train shape:", example["X_train"].shape)
print("X_pred shape:", example["X_pred"].shape)
print("Y_train shape:", example["Y_train"].shape)
print("y_real shape:", example["y_real"].shape)

# Estatísticas da padronização na janela (só para sanity check)
X_train = example["X_train"]
print("\nMédia aproximada das RFF padronizadas na janela (deve estar perto de 0):",
      np.mean(X_train))
print("Desvio padrão aproximado das RFF padronizadas na janela (deve estar perto de 1):",
      np.std(X_train))

# ============================================================
# A partir daqui, o próximo passo é plugar o Ridge:
#   para cada window_idx em range(window_map.shape[0]):
#       data = prepare_window_data(...)
#       usar data["X_train"], data["Y_train"], data["X_pred"] no Ridge
#       e armazenar as previsões + y_real para avaliar a estratégia
# ============================================================

df_St: 110 meses × 12000 RFFs
ex_R_acoes_tmais1_treino: 110 meses × 114 ações
window_map com 97 janelas de treino (previsões OOS)

=== Exemplo da primeira janela ===
Data_pred: 2016-11-30 00:00:00
idx_pred: 12
train_slice (start, end): (0, 11)
X_train shape: (12, 12000)
X_pred shape: (1, 12000)
Y_train shape: (12, 114)
y_real shape: (1, 114)

Média aproximada das RFF padronizadas na janela (deve estar perto de 0): 4.0399782284970975e-19
Desvio padrão aproximado das RFF padronizadas na janela (deve estar perto de 1): 1.0


# **5. Estimando os $\beta$'s (Ridge Regression)**

In [161]:
import numpy as np
import pandas as pd
from pathlib import Path
import os

# ============================================================
# 1) Parâmetro de shrinkage e pasta de saída
# ============================================================

z = 0.1  # parâmetro de shrinkage fixo (L2)
betas_dir = Path("Betas_Ridge_z10")
os.makedirs(betas_dir, exist_ok=True)

print(f"Shrinkage z = {z}")
print(f"Pasta para salvar betas: {betas_dir.resolve()}")

# ============================================================
# 2) Função de Ridge (forma dual, sem intercepto)
# ============================================================

def ridge_fit_window(X_train: np.ndarray, Y_train: np.ndarray, z: float) -> np.ndarray:
    """
    Ajusta Ridge multi-output: Y = X B + erro
    X_train: (T × P)
    Y_train: (T × N)
    z      : shrinkage L2

    Retorna:
      B_hat: (P × N)
    """
    T, P = X_train.shape
    _, N = Y_train.shape

    # Matriz T×T (XX') e termo de regularização
    XXt = X_train @ X_train.T          # (T × T)
    A = XXt + z * T * np.eye(T)        # (T × T)

    # Resolver A * Alpha = Y para Alpha (T × N)
    # Em vez de inverter A, usamos solve
    try:
        Alpha = np.linalg.solve(A, Y_train)  # (T × N)
    except np.linalg.LinAlgError:
        # Em caso de problemas numéricos, recorre a pseudo-inversa
        Alpha = np.linalg.pinv(A) @ Y_train

    # B_hat = X' * Alpha / T  (P × N)
    B_hat = (X_train.T @ Alpha) / T
    return B_hat

# ============================================================
# 3) Loop sobre todas as janelas de treino e salvar betas
# ============================================================

records = []

n_windows = window_map.shape[0]
print(f"Iniciando ajuste Ridge em {n_windows} janelas...")

for window_idx in range(n_windows):
    # Dados da janela (já com RFF padronizadas!)
    data = prepare_window_data(
        df_St=df_St,
        ex_R_t1=ex_R_acoes_tmais1_treino,
        window_map=window_map,
        window_idx=window_idx,
        rff_cols=rff_cols,
        action_cols=action_cols
    )

    X_train = data["X_train"]    # (12 × 12000)
    Y_train = data["Y_train"]    # (12 × 114)

    # Ajusta Ridge e obtém B_hat (12000 × 114)
    B_hat = ridge_fit_window(X_train, Y_train, z=z)

    # Converter para float32 para economizar espaço em disco
    B_hat32 = B_hat.astype(np.float32)

    # Nome do arquivo de betas desta janela
    beta_filename = f"betas_ridge_z{int(z)}_window_{window_idx:03d}.npy"
    beta_path = betas_dir / beta_filename

    # Salvar matriz de betas
    np.save(beta_path, B_hat32)

    # Guardar metadados da janela
    idx_pred = int(data["idx_pred"])
    idx_train_start, idx_train_end = data["train_slice"]

    records.append({
        "window_idx": window_idx,
        "Data_pred": data["Data_pred"],
        "idx_pred": idx_pred,
        "idx_train_start": idx_train_start,
        "idx_train_end": idx_train_end,
        "beta_file": beta_filename,
        "z": z
    })

    if (window_idx + 1) % 10 == 0 or window_idx == n_windows - 1:
        print(f"Janela {window_idx+1}/{n_windows} concluída.")

# ============================================================
# 4) Salvar metadados das janelas e checar
# ============================================================

betas_meta = pd.DataFrame(records)
meta_path = betas_dir / "betas_ridge_metadata_z10.csv"
betas_meta.to_csv(meta_path, index=False)

print("\nAjuste Ridge concluído.")
print(f"Arquivos de betas salvos em: {betas_dir.resolve()}")
print(f"Metadados salvos em: {meta_path.resolve()}")

print("\nPré-visualização dos metadados:")
print(betas_meta.head())
print(f"\nTotal de janelas com betas salvos: {betas_meta.shape[0]}")

Shrinkage z = 0.1
Pasta para salvar betas: C:\Users\Hugo Villanova\Desktop\DQAI2025\Metodologia - V3\Betas_Ridge_z10
Iniciando ajuste Ridge em 97 janelas...
Janela 10/97 concluída.
Janela 20/97 concluída.
Janela 30/97 concluída.
Janela 40/97 concluída.
Janela 50/97 concluída.
Janela 60/97 concluída.
Janela 70/97 concluída.
Janela 80/97 concluída.
Janela 90/97 concluída.
Janela 97/97 concluída.

Ajuste Ridge concluído.
Arquivos de betas salvos em: C:\Users\Hugo Villanova\Desktop\DQAI2025\Metodologia - V3\Betas_Ridge_z10
Metadados salvos em: C:\Users\Hugo Villanova\Desktop\DQAI2025\Metodologia - V3\Betas_Ridge_z10\betas_ridge_metadata_z10.csv

Pré-visualização dos metadados:
   window_idx  Data_pred  idx_pred  idx_train_start  idx_train_end  \
0           0 2016-11-30        12                0             11   
1           1 2016-12-31        13                1             12   
2           2 2017-01-31        14                2             13   
3           3 2017-02-28        15    

# **6. Previsões OOS $\hat{R_{t+1}}$**

In [162]:
import numpy as np
import pandas as pd
from pathlib import Path

# ============================================================
# 1) Caminhos dos arquivos
# ============================================================

st_path_parquet    = Path("St_RFF_12000_g2_seed123.parquet")
st_path_csv        = Path("St_RFF_12000_g2_seed123.csv")

ex_t1_path_parquet = Path("ex_R_acoes_tmais1_treino.parquet")
ex_t1_path_csv     = Path("ex_R_acoes_tmais1_treino.csv")

window_map_parquet = Path("window_map_12m.parquet")
window_map_csv     = Path("window_map_12m.csv")

betas_dir          = Path("Betas_Ridge_z10")
betas_meta_path    = betas_dir / "betas_ridge_metadata_z10.csv"

# ============================================================
# 2) Carregar bases principais
# ============================================================

# --- df_St: RFF brutas ---
if st_path_parquet.exists():
    df_St = pd.read_parquet(st_path_parquet)
elif st_path_csv.exists():
    df_St = pd.read_csv(st_path_csv, parse_dates=["Data"])
else:
    raise FileNotFoundError("Não encontrei df_St (parquet/csv).")

assert "Data" in df_St.columns, "df_St precisa ter coluna 'Data'."
df_St = df_St.sort_values("Data").reset_index(drop=True)

# --- ex_R_acoes_tmais1_treino: excessos t+1 ---
if ex_t1_path_parquet.exists():
    ex_R_acoes_tmais1_treino = pd.read_parquet(ex_t1_path_parquet)
elif ex_t1_path_csv.exists():
    ex_R_acoes_tmais1_treino = pd.read_csv(ex_t1_path_csv, parse_dates=["Data"])
else:
    raise FileNotFoundError("Não encontrei ex_R_acoes_tmais1_treino (parquet/csv).")

assert "Data" in ex_R_acoes_tmais1_treino.columns, "ex_R_acoes_tmais1_treino precisa ter coluna 'Data'."
ex_R_acoes_tmais1_treino = ex_R_acoes_tmais1_treino.sort_values("Data").reset_index(drop=True)

# --- window_map: índices de treino e previsão ---
if window_map_parquet.exists():
    window_map = pd.read_parquet(window_map_parquet)
elif window_map_csv.exists():
    window_map = pd.read_csv(window_map_csv, parse_dates=["Data_pred"])
else:
    raise FileNotFoundError("Não encontrei window_map_12m (parquet/csv).")

# --- metadados dos betas ---
if not betas_meta_path.exists():
    raise FileNotFoundError(f"Metadados de betas não encontrados em {betas_meta_path}")

betas_meta = pd.read_csv(betas_meta_path, parse_dates=["Data_pred"])

# ============================================================
# 3) Checagens básicas e definição de colunas
# ============================================================

# Verificar alinhamento de datas entre df_St e ex_R_acoes_tmais1_treino
if not df_St["Data"].equals(ex_R_acoes_tmais1_treino["Data"]):
    raise ValueError("As datas de df_St e ex_R_acoes_tmais1_treino não estão alinhadas 1:1.")

datas = df_St["Data"].copy()

# Colunas de RFF (features) e ações (targets)
rff_cols = [c for c in df_St.columns if c.startswith("RFF_")]
action_cols = [c for c in ex_R_acoes_tmais1_treino.columns if c != "Data"]

print(f"df_St: {df_St.shape[0]} meses × {len(rff_cols)} RFFs")
print(f"ex_R_acoes_tmais1_treino: {ex_R_acoes_tmais1_treino.shape[0]} meses × {len(action_cols)} ações")
print(f"window_map: {window_map.shape[0]} janelas")
print(f"betas_meta: {betas_meta.shape[0]} janelas")

# Garantir que window_map e betas_meta têm mesma quantidade de janelas
if window_map.shape[0] != betas_meta.shape[0]:
    raise ValueError("window_map e betas_meta têm números de janelas diferentes.")

# ============================================================
# 4) (Re)definir prepare_window_data se não estiver no escopo
#    (SE você já tiver ela no notebook, pode pular esta definição)
# ============================================================

def prepare_window_data(
    df_St: pd.DataFrame,
    ex_R_t1: pd.DataFrame,
    window_map: pd.DataFrame,
    window_idx: int,
    rff_cols: list,
    action_cols: list,
    std_epsilon: float = 1e-8
):
    row = window_map.iloc[window_idx]
    idx_pred = int(row["idx_pred"])
    idx_train_start = int(row["idx_train_start"])
    idx_train_end = int(row["idx_train_end"])
    data_pred = row["Data_pred"]

    # Features bruto
    S_window_raw = df_St.loc[idx_train_start:idx_train_end, rff_cols].to_numpy(dtype=float)
    S_pred_raw   = df_St.loc[[idx_pred], rff_cols].to_numpy(dtype=float)

    # Targets (t+1) bruto
    Y_window = ex_R_t1.loc[idx_train_start:idx_train_end, action_cols].to_numpy(dtype=float)
    y_real   = ex_R_t1.loc[[idx_pred], action_cols].to_numpy(dtype=float)

    # Padronização das RFF na janela
    mean = S_window_raw.mean(axis=0)
    std  = S_window_raw.std(axis=0, ddof=0)
    std_safe = np.where(std < std_epsilon, 1.0, std)

    X_train = (S_window_raw - mean) / std_safe
    X_pred  = (S_pred_raw  - mean) / std_safe

    return {
        "Data_pred": data_pred,
        "idx_pred": idx_pred,
        "train_slice": (idx_train_start, idx_train_end),
        "X_train": X_train,
        "X_pred": X_pred,
        "Y_train": Y_window,
        "y_real": y_real,
        "rff_cols": rff_cols,
        "action_cols": action_cols,
    }

# ============================================================
# 5) Loop para gerar previsões OOS (sem ranking)
# ============================================================

all_preds = []   # para guardar previsões
all_real  = []   # para guardar realizados, se quiser

n_windows = window_map.shape[0]
print(f"Iniciando geração de previsões para {n_windows} janelas...")

for window_idx in range(n_windows):
    # Dados da janela (X_pred padronizado + y_real)
    data = prepare_window_data(
        df_St=df_St,
        ex_R_t1=ex_R_acoes_tmais1_treino,
        window_map=window_map,
        window_idx=window_idx,
        rff_cols=rff_cols,
        action_cols=action_cols
    )

    X_pred = data["X_pred"]   # (1 × 12000)
    y_real = data["y_real"]   # (1 × 114)
    data_pred = data["Data_pred"]

    # Arquivo de betas correspondente à janela
    beta_file = betas_meta.loc[betas_meta["window_idx"] == window_idx, "beta_file"].iloc[0]
    beta_path = betas_dir / beta_file

    if not beta_path.exists():
        raise FileNotFoundError(f"Arquivo de betas não encontrado: {beta_path}")

    # Carregar B_hat (12000 × 114)
    B_hat = np.load(beta_path)  # foi salvo em float32

    # Previsão: y_hat = X_pred @ B_hat  → shape (1 × 114)
    y_hat = X_pred @ B_hat
    y_hat = y_hat.reshape(-1)   # vetor 1D com 114 elementos
    y_real_vec = y_real.reshape(-1)

    # Montar dict com previsões e realizados
    pred_row = {"Data_pred": data_pred}
    real_row = {"Data_pred": data_pred}

    # Preencher com colunas por ação (mesma ordem de action_cols)
    for col_name, val_pred, val_real in zip(action_cols, y_hat, y_real_vec):
        pred_row[col_name] = val_pred
        real_row[col_name] = val_real

    all_preds.append(pred_row)
    all_real.append(real_row)

    if (window_idx + 1) % 10 == 0 or window_idx == n_windows - 1:
        print(f"Janela {window_idx+1}/{n_windows} processada.")

# ============================================================
# 6) Construir DataFrames de previsões e realizados e salvar
# ============================================================

df_preds = pd.DataFrame(all_preds).sort_values("Data_pred").reset_index(drop=True)
df_real  = pd.DataFrame(all_real).sort_values("Data_pred").reset_index(drop=True)

print("\nPré-visualização das previsões:")
print(df_preds.head())

print("\nPré-visualização dos realizados (excessos t+1):")
print(df_real.head())

print(f"\ndf_preds shape: {df_preds.shape} (meses OOS × (1 + {len(action_cols)} ações))")
print(f"df_real  shape: {df_real.shape}")

# Salvar em disco
preds_path_parquet = Path("predicoes_excesso_ridge_z10.parquet")
real_path_parquet  = Path("reais_excesso_ridge_z10.parquet")

df_preds.to_parquet(preds_path_parquet, index=False)
df_real.to_parquet(real_path_parquet, index=False)

# (Opcional) também em CSV
df_preds.to_csv("predicoes_excesso_ridge_z10.csv", index=False)
df_real.to_csv("reais_excesso_ridge_z10.csv", index=False)

print(f"\nPrevisões salvas em: {preds_path_parquet.resolve()}")
print(f"Realizados salvos em: {real_path_parquet.resolve()}")

df_St: 110 meses × 12000 RFFs
ex_R_acoes_tmais1_treino: 110 meses × 114 ações
window_map: 97 janelas
betas_meta: 97 janelas
Iniciando geração de previsões para 97 janelas...
Janela 10/97 processada.
Janela 20/97 processada.
Janela 30/97 processada.
Janela 40/97 processada.
Janela 50/97 processada.
Janela 60/97 processada.
Janela 70/97 processada.
Janela 80/97 processada.
Janela 90/97 processada.
Janela 97/97 processada.

Pré-visualização das previsões:
   Data_pred     ABCB4     ABEV3     AGRO3     ALPA4    ALUP11     AMER3  \
0 2016-11-30  0.000110 -0.000081  0.000350  0.000099  0.000182  0.000390   
1 2016-12-31  0.000097  0.000126 -0.000173 -0.000406 -0.000041  0.000767   
2 2017-01-31  0.000147 -0.000020 -0.000030 -0.000132  0.000307  0.000462   
3 2017-02-28 -0.000061 -0.000055 -0.000113  0.000040 -0.000370 -0.000242   
4 2017-03-31 -0.000195  0.000103  0.000046 -0.000117 -0.000166  0.000151   

      ANIM3     AZZA3     B3SA3  ...     TUPY3     UCAS3     UGPA3     UNIP6  \
0  0.0

# **7. Ranking Long-Short**

In [163]:
import pandas as pd
from pathlib import Path

# ===========================================
# 1) Carregar previsões e realizados
# ===========================================
preds_parquet = Path("predicoes_excesso_ridge_z10.parquet")
preds_csv     = Path("predicoes_excesso_ridge_z10.csv")

real_parquet  = Path("reais_excesso_ridge_z10.parquet")
real_csv      = Path("reais_excesso_ridge_z10.csv")

# df_preds: Data_pred + 114 colunas de previsões
if preds_parquet.exists():
    df_preds = pd.read_parquet(preds_parquet)
elif preds_csv.exists():
    df_preds = pd.read_csv(preds_csv, parse_dates=["Data_pred"])
else:
    raise FileNotFoundError("Não encontrei predicoes_excesso_ridge_z10 (parquet/csv).")

# df_real: Data_pred + 114 colunas de realizados
if real_parquet.exists():
    df_real = pd.read_parquet(real_parquet)
elif real_csv.exists():
    df_real = pd.read_csv(real_csv, parse_dates=["Data_pred"])
else:
    raise FileNotFoundError("Não encontrei reais_excesso_ridge_z10 (parquet/csv).")

# Garantir que as datas estão alinhadas
df_preds = df_preds.sort_values("Data_pred").reset_index(drop=True)
df_real  = df_real.sort_values("Data_pred").reset_index(drop=True)

if not df_preds["Data_pred"].equals(df_real["Data_pred"]):
    raise ValueError("Datas de df_preds e df_real não estão alinhadas.")

# Colunas de ações (todas menos Data_pred)
action_cols = [c for c in df_preds.columns if c != "Data_pred"]

print(f"Meses OOS: {df_preds.shape[0]}, número de ações: {len(action_cols)}")

# ===========================================
# 2) Para cada mês, selecionar top 10 e bottom 10
# ===========================================
rows = []

for i in range(df_preds.shape[0]):
    data_pred = df_preds.loc[i, "Data_pred"]
    
    # Séries de previsões e realizados nesse mês (índice = ticker)
    preds_row = df_preds.loc[i, action_cols]
    real_row  = df_real.loc[i, action_cols]
    
    # Ordenar por previsão (descrescente)
    preds_sorted = preds_row.sort_values(ascending=False)
    
    # Top 10 LONG
    longs = preds_sorted.index[:10]
    # Bottom 10 SHORT
    shorts = preds_sorted.index[-10:]
    
    # Registrar LONGs
    for ticker in longs:
        rows.append({
            "Data_pred": data_pred,
            "Ticker": ticker,
            "Side": "LONG",
            "Pred_excess": preds_row[ticker],
            "Real_excess": real_row[ticker],
        })
    
    # Registrar SHORTs
    for ticker in shorts:
        rows.append({
            "Data_pred": data_pred,
            "Ticker": ticker,
            "Side": "SHORT",
            "Pred_excess": preds_row[ticker],
            "Real_excess": real_row[ticker],
        })

# ===========================================
# 3) Construir DataFrame final e salvar em xlsx
# ===========================================
df_signals = pd.DataFrame(rows)

# Ordenar por data e, dentro da data, LONGs em cima, depois SHORTs
side_order = {"LONG": 0, "SHORT": 1}
df_signals["Side_order"] = df_signals["Side"].map(side_order)

df_signals = (
    df_signals
    .sort_values(["Data_pred", "Side_order", "Ticker"])
    .drop(columns=["Side_order"])
    .reset_index(drop=True)
)

print(df_signals.head())
print(f"\nTotal de linhas (meses × 20 ativos): {df_signals.shape[0]}")

# Salvar em Excel
output_path = Path("sinais_long_short_ridge_z10.xlsx")
df_signals.to_excel(output_path, index=False)

print(f"\nArquivo Excel salvo em: {output_path.resolve()}")

Meses OOS: 97, número de ações: 114
   Data_pred  Ticker  Side  Pred_excess  Real_excess
0 2016-11-30   BRAP4  LONG     0.000717    -0.010085
1 2016-11-30   CSNA3  LONG     0.000681    -0.147103
2 2016-11-30   GOAU4  LONG     0.000870    -0.207270
3 2016-11-30  GOLL54  LONG     0.001079    -0.170856
4 2016-11-30   OIBR3  LONG     0.000699    -0.055772

Total de linhas (meses × 20 ativos): 1940

Arquivo Excel salvo em: C:\Users\Hugo Villanova\Desktop\DQAI2025\Metodologia - V3\sinais_long_short_ridge_z10.xlsx


# **8. Retornos Finais**

In [164]:
import numpy as np
import pandas as pd
from pathlib import Path

# ====================================================
# 1) Caminhos dos arquivos
# ====================================================

base_path   = Path("Base_Final_DQAI2025_v3.xlsx")
sinais_path = Path("sinais_long_short_ridge_z10.xlsx")

# Parâmetro de custo de transação
# custo por unidade de exposição bruta (long + short) por rebalance
TX_COST_RATE = 0.0005  # 5 bps
GROSS_EXPOSURE = 2.0   # 100% long + 100% short => exposição bruta = 2
TX_COST_SIMPLE = TX_COST_RATE * GROSS_EXPOSURE  # custo simples aplicado no 1º dia útil de t+1

print(f"Custo de transação por rebalance (simples): {TX_COST_SIMPLE:.4%}")

# ====================================================
# 2) Carregar sinais mensais (LONG/SHORT)
# ====================================================

df_sinais = pd.read_excel(sinais_path)
df_sinais["Data_pred"] = pd.to_datetime(df_sinais["Data_pred"])

# Garantir ordenação por data
df_sinais = df_sinais.sort_values("Data_pred").reset_index(drop=True)

print(df_sinais.head())

# ====================================================
# 3) Carregar log-retornos diários das ações e CDI
# ====================================================

# Ações - log retornos diários
df_retD = pd.read_excel(base_path, sheet_name="Acoes Log Ret D")
df_retD["Data"] = pd.to_datetime(df_retD["Data"], dayfirst=True, errors="coerce")
df_retD = df_retD.dropna(subset=["Data"]).sort_values("Data").reset_index(drop=True)

# CDI - log retornos diários
df_cdiD = pd.read_excel(base_path, sheet_name="CDI Log Ret D")
df_cdiD["Data"] = pd.to_datetime(df_cdiD["Data"], dayfirst=True, errors="coerce")
df_cdiD = df_cdiD.dropna(subset=["Data"]).sort_values("Data").reset_index(drop=True)

# Identificar coluna de CDI
cdi_cols = [c for c in df_cdiD.columns if c != "Data"]
assert len(cdi_cols) >= 1, "Sheet 'CDI Log Ret D' deve ter pelo menos 1 coluna além de 'Data'."
cdi_col = cdi_cols[0]

# Criar coluna de Year-Month (período) para facilitar filtro por mês
df_retD["YM"] = df_retD["Data"].dt.to_period("M")
df_cdiD["YM"] = df_cdiD["Data"].dt.to_period("M")

# Lista de tickers disponíveis na base diária
tickers_diarios = [c for c in df_retD.columns if c not in ["Data", "YM"]]

print(f"Datas diárias de ações: {df_retD['Data'].min().date()} → {df_retD['Data'].max().date()}")
print(f"Datas diárias de CDI:   {df_cdiD['Data'].min().date()} → {df_cdiD['Data'].max().date()}")
print(f"Número de ações na base diária: {len(tickers_diarios)}")

# ====================================================
# 4) Loop por mês OOS: montar retornos diários da estratégia
# ====================================================

rows_out = []

# Agrupar sinais por Data_pred (cada mês OOS)
for data_pred, grupo in df_sinais.groupby("Data_pred"):
    # Mês "alvo" é t+1 (mês seguinte ao Data_pred)
    ym_pred = data_pred.to_period("M")
    ym_target = ym_pred + 1  # mês t+1

    # Filtrar datas diárias no mês t+1
    mask_ret = df_retD["YM"] == ym_target
    df_mes_ret = df_retD.loc[mask_ret].copy()

    mask_cdi = df_cdiD["YM"] == ym_target
    df_mes_cdi = df_cdiD.loc[mask_cdi, ["Data", cdi_col]].copy()

    if df_mes_ret.empty or df_mes_cdi.empty:
        print(f"Atenção: sem dados diários para o mês {ym_target} (Data_pred = {data_pred.date()}).")
        continue

    # Garantir alinhamento de dias entre ações e CDI
    df_mes_ret = df_mes_ret.set_index("Data")
    df_mes_cdi = df_mes_cdi.set_index("Data")
    df_mes = df_mes_ret.join(df_mes_cdi, how="inner").sort_index()

    if df_mes.empty:
        print(f"Atenção: sem interseção de dias entre ações e CDI para o mês {ym_target}.")
        continue

    # Tickers LONG e SHORT nesse Data_pred
    longs  = grupo.loc[grupo["Side"] == "LONG", "Ticker"].tolist()
    shorts = grupo.loc[grupo["Side"] == "SHORT", "Ticker"].tolist()

    if len(longs) != 10 or len(shorts) != 10:
        print(f"Atenção: Data_pred = {data_pred.date()} não tem 10 LONG e 10 SHORT (LONG={len(longs)}, SHORT={len(shorts)}).")

    longs_valid  = [t for t in longs if t in df_mes.columns]
    shorts_valid = [t for t in shorts if t in df_mes.columns]

    if len(longs_valid) == 0 or len(shorts_valid) == 0:
        print(f"Atenção: nenhum ticker válido em LONG ou SHORT para Data_pred = {data_pred.date()}. Pulando.")
        continue

    # ----------------------------------------------------
    # 4.1. Converter log-retornos das ações para simples
    # ----------------------------------------------------
    # Long leg
    log_ret_long_assets = df_mes[longs_valid]              # DataFrame (dias × n_long)
    r_long_assets = np.exp(log_ret_long_assets) - 1.0      # simples

    # Short leg
    log_ret_short_assets = df_mes[shorts_valid]            # DataFrame (dias × n_short)
    r_short_assets = np.exp(log_ret_short_assets) - 1.0    # simples

    # ----------------------------------------------------
    # 4.2. Retornos simples das carteiras LONG e SHORT
    #      (equal-weight: média simples das ações em cada dia)
    # ----------------------------------------------------
    r_long = r_long_assets.mean(axis=1)            # série diária (simples)
    r_short_underlying = r_short_assets.mean(axis=1)

    # Perna short: lucramos quando o ativo cai ⇒ retorno = -retorno do ativo
    r_short = -r_short_underlying

    # CDI em retorno simples
    r_cdi_simple = np.exp(df_mes[cdi_col]) - 1.0

    # ----------------------------------------------------
    # 4.3. Retorno simples TOTAL da estratégia:
    #      Long + Short + CDI (caixa do short rendendo CDI)
    # ----------------------------------------------------
    r_total_bruto = r_long + r_short + r_cdi_simple

    # ----------------------------------------------------
    # 4.4. Aplicar custo de transação no primeiro dia útil do mês t+1
    # ----------------------------------------------------
    r_total_liq = r_total_bruto.copy()

    # Índice do primeiro dia útil desse mês (primeira linha de df_mes)
    first_day = df_mes.index[0]
    # Aplicar custo simples nesse dia (subtraindo TX_COST_SIMPLE)
    r_total_liq.loc[first_day] = r_total_liq.loc[first_day] - TX_COST_SIMPLE

    # ----------------------------------------------------
    # 4.5. Converter tudo para log-retornos
    # ----------------------------------------------------
    ret_long_log      = np.log1p(r_long)
    ret_short_log     = np.log1p(r_short)
    ret_total_bruto_log = np.log1p(r_total_bruto)
    ret_total_liq_log   = np.log1p(r_total_liq)
    cdi_log           = df_mes[cdi_col]  # já em log

    # ----------------------------------------------------
    # 4.6. Guardar resultados dia a dia
    # ----------------------------------------------------
    for data_dia in df_mes.index:
        rows_out.append({
            "Data_dia": data_dia,
            "Data_pred": data_pred,         # mês em que a carteira foi formada
            "Mes_t1": ym_target,            # mês de referência dos retornos
            "Ret_log_LONG":     ret_long_log.loc[data_dia],
            "Ret_log_SHORT":    ret_short_log.loc[data_dia],
            "Ret_log_TOTAL_bruto": ret_total_bruto_log.loc[data_dia],
            "Ret_log_TOTAL_liq":   ret_total_liq_log.loc[data_dia],
            f"Ret_log_{cdi_col}": cdi_log.loc[data_dia],
        })

# ====================================================
# 5) Construir DataFrame final e salvar em Excel
# ====================================================

df_ret_diario = pd.DataFrame(rows_out)
df_ret_diario = (
    df_ret_diario
    .sort_values(["Data_dia", "Data_pred"])
    .reset_index(drop=True)
)

print(df_ret_diario.head())
print(f"\nTotal de dias na série da estratégia: {df_ret_diario.shape[0]}")

# Salvar em Excel
output_path = Path("retornos_diarios_estrategia_z10_comCDI_e_custo.xlsx")
df_ret_diario.to_excel(output_path, index=False)

print(f"\nArquivo de retornos diários salvo em: {output_path.resolve()}")

Custo de transação por rebalance (simples): 0.1000%
   Data_pred Ticker   Side  Pred_excess  Real_excess
0 2016-11-30  BRAP4   LONG     0.000717    -0.010085
1 2016-11-30  WEGE3  SHORT    -0.000073    -0.010278
2 2016-11-30  TOTS3  SHORT    -0.000059     0.110706
3 2016-11-30  SMTO3  SHORT    -0.000252     0.071484
4 2016-11-30  SIMH3  SHORT    -0.000311    -0.088281
Datas diárias de ações: 2013-12-02 → 2024-12-30
Datas diárias de CDI:   2013-12-02 → 2024-12-30
Número de ações na base diária: 281
    Data_dia  Data_pred   Mes_t1  Ret_log_LONG  Ret_log_SHORT  \
0 2016-12-01 2016-11-30  2016-12     -0.042912       0.030719   
1 2016-12-02 2016-11-30  2016-12      0.000579      -0.002598   
2 2016-12-05 2016-11-30  2016-12     -0.003114       0.011240   
3 2016-12-06 2016-11-30  2016-12      0.021293      -0.021410   
4 2016-12-07 2016-11-30  2016-12      0.004438      -0.003579   

   Ret_log_TOTAL_bruto  Ret_log_TOTAL_liq  Ret_log_CDI_LogRet_D  
0            -0.010355          -0.011366

In [165]:
import numpy as np
import pandas as pd
from pathlib import Path

# ==========================================
# 1) Carregar retornos diários da estratégia
# ==========================================

ret_diario_path = Path("retornos_diarios_estrategia_z10_comCDI_e_custo.xlsx")
df_ret = pd.read_excel(ret_diario_path)

# Garantir tipo de data
df_ret["Data_dia"] = pd.to_datetime(df_ret["Data_dia"])

# Filtrar a partir de 2017-01-01
df_ret = df_ret[df_ret["Data_dia"] >= pd.Timestamp("2017-01-01")].copy()
df_ret = df_ret.sort_values("Data_dia").reset_index(drop=True)

print("Período filtrado:")
print(df_ret["Data_dia"].min(), "→", df_ret["Data_dia"].max())

# ==========================================
# 2) Identificar colunas de retorno log
# ==========================================

# Colunas que já sabemos
col_long   = "Ret_log_LONG"
col_short  = "Ret_log_SHORT"
col_total_b = "Ret_log_TOTAL_bruto"
col_total_l = "Ret_log_TOTAL_liq"

# Encontrar coluna do CDI (é a outra que começa com Ret_log_ e não é as de cima)
known_cols = {col_long, col_short, col_total_b, col_total_l}
cdi_cols = [c for c in df_ret.columns if c.startswith("Ret_log_") and c not in known_cols]
assert len(cdi_cols) == 1, f"Esperava encontrar 1 coluna de CDI em log, encontrei: {cdi_cols}"
col_cdi = cdi_cols[0]

print("Colunas de retornos log usadas:")
print("LONG:", col_long)
print("SHORT:", col_short)
print("TOTAL_bruto:", col_total_b)
print("TOTAL_liq:", col_total_l)
print("CDI:", col_cdi)

# ==========================================
# 3) Função auxiliar para acumular retornos
# ==========================================

def add_cumulative_columns(df: pd.DataFrame, col_log: str, prefix: str):
    """
    A partir de uma coluna de log-retorno diário (col_log), adiciona:

      - f"{prefix}_cum_log"      : soma cumulativa dos log-retornos
      - f"{prefix}_cum_simple"   : retorno simples cumulativo = exp(cum_log) - 1
    """
    cum_log_col = f"{prefix}_cum_log"
    cum_simple_col = f"{prefix}_cum_simple"

    df[cum_log_col] = df[col_log].cumsum()
    df[cum_simple_col] = np.exp(df[cum_log_col]) - 1.0

    return df

# ==========================================
# 4) Adicionar retornos acumulados para cada perna
# ==========================================

df_acum = df_ret.copy()

# LONG
df_acum = add_cumulative_columns(df_acum, col_long, "LONG")

# SHORT
df_acum = add_cumulative_columns(df_acum, col_short, "SHORT")

# CDI
df_acum = add_cumulative_columns(df_acum, col_cdi, "CDI")

# Estratégia - vamos usar a perna TOTAL líquida (já com custo)
df_acum = add_cumulative_columns(df_acum, col_total_l, "ESTRAT_LIQ")

# (Opcional) se quiser também a versão bruta:
# df_acum = add_cumulative_columns(df_acum, col_total_b, "ESTRAT_BRUTA")

# ==========================================
# 5) Ver uma amostra e salvar
# ==========================================

cols_view = [
    "Data_dia",
    col_long,
    "LONG_cum_log", "LONG_cum_simple",
    col_short,
    "SHORT_cum_log", "SHORT_cum_simple",
    col_cdi,
    "CDI_cum_log", "CDI_cum_simple",
    col_total_l,
    "ESTRAT_LIQ_cum_log", "ESTRAT_LIQ_cum_simple",
]

print(df_acum[cols_view].head())

output_path = Path("retornos_acumulados_estrategia_z10_desde2017.xlsx")
df_acum.to_excel(output_path, index=False)

print(f"\nArquivo com retornos acumulados salvo em: {output_path.resolve()}")

Período filtrado:
2017-01-02 00:00:00 → 2024-12-30 00:00:00
Colunas de retornos log usadas:
LONG: Ret_log_LONG
SHORT: Ret_log_SHORT
TOTAL_bruto: Ret_log_TOTAL_bruto
TOTAL_liq: Ret_log_TOTAL_liq
CDI: Ret_log_CDI_LogRet_D
    Data_dia  Ret_log_LONG  LONG_cum_log  LONG_cum_simple  Ret_log_SHORT  \
0 2017-01-02      0.040557      0.040557         0.041391       0.008176   
1 2017-01-03      0.028499      0.069056         0.071496      -0.036909   
2 2017-01-04      0.014051      0.083107         0.086658      -0.011948   
3 2017-01-05      0.023035      0.106143         0.111980      -0.017725   
4 2017-01-06     -0.000006      0.106136         0.111973       0.009275   

   SHORT_cum_log  SHORT_cum_simple  Ret_log_CDI_LogRet_D  CDI_cum_log  \
0       0.008176          0.008209              0.000507     0.000507   
1      -0.028733         -0.028324              0.000507     0.001014   
2      -0.040682         -0.039865              0.000507     0.001521   
3      -0.058407         -0.056